In [1]:
# to prevent unnecessary warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess
import cv2
import json
import requests
from tqdm import tqdm

tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

%matplotlib inline

1.15.0


In [4]:
### Loading MNIST DIGITS dataset

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
assert x_train.shape == (60000, 28, 28)
assert x_test.shape == (10000, 28, 28)
assert y_train.shape == (60000,)
assert y_test.shape == (10000,)

class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

print('\nx_train.shape: {}, of {}'.format(x_train.shape, x_train.dtype))
print('x_test.shape: {}, of {}'.format(x_test.shape, x_test.dtype))


x_train.shape: (60000, 28, 28), of uint8
x_test.shape: (10000, 28, 28), of uint8


In [5]:
### Reshape the data

x_train_gr = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test_gr = x_test.reshape(x_test.shape[0], 28, 28, 1)

print('\nTrain_images.shape: {}, of {}'.format(x_train_gr.shape, x_train_gr.dtype))
print('Test_images.shape: {}, of {}'.format(x_test_gr.shape, x_test_gr.dtype))


Train_images.shape: (60000, 28, 28, 1), of uint8
Test_images.shape: (10000, 28, 28, 1), of uint8


In [6]:
### Design the CNN model

INPUT_SHAPE = (28, 28, 1)

def create_cnn_architecture_model1(input_shape):
    inp = keras.layers.Input(shape=input_shape)

    conv1 = keras.layers.Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same')(inp)
    pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same')(pool1)
    pool2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    flat = keras.layers.Flatten()(pool2)

    hidden1 = keras.layers.Dense(256, activation='relu')(flat)
    drop1 = keras.layers.Dropout(rate=0.3)(hidden1)

    out = keras.layers.Dense(10, activation='softmax')(drop1)

    model = keras.Model(inputs=inp, outputs=out)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model
    
model = create_cnn_architecture_model1(input_shape=INPUT_SHAPE)
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1568)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               401664

In [9]:
### Training the CNN model

# EPOCHS = 10
# x_train_scaled = x_train_gr / 255.
# model.fit(x_train_scaled, y_train, validation_split=0.1, epochs=EPOCHS)

Train on 54000 samples, validate on 6000 samples
Epoch 1/10
54000/54000 [==============================] - 12s 221us/sample - loss: 0.1591 - acc: 0.9518 - val_loss: 0.0521 - val_acc: 0.9858
Epoch 2/10
54000/54000 [==============================] - 12s 219us/sample - loss: 0.0548 - acc: 0.9826 - val_loss: 0.0451 - val_acc: 0.9868
Epoch 3/10
54000/54000 [==============================] - 12s 218us/sample - loss: 0.0388 - acc: 0.9877 - val_loss: 0.0397 - val_acc: 0.9893
Epoch 4/10
54000/54000 [==============================] - 12s 221us/sample - loss: 0.0311 - acc: 0.9904 - val_loss: 0.0311 - val_acc: 0.9920
Epoch 5/10
54000/54000 [==============================] - 12s 222us/sample - loss: 0.0228 - acc: 0.9926 - val_loss: 0.0350 - val_acc: 0.9905
Epoch 6/10
54000/54000 [==============================] - 12s 221us/sample - loss: 0.0191 - acc: 0.9938 - val_loss: 0.0348 - val_acc: 0.9903
Epoch 7/10
54000/54000 [==============================] - 12s 226us/sample - loss: 0.0177 - acc: 0.9943 -

In [14]:
### Classification

import os
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd

# save model
if not os.path.isdir('model_weights/'):
    os.mkdir('model_weights/')
model.save_weights(filepath='model_weights/cnn_mnist_digits_wt.h5', overwrite=True)

# load model (can be used in the future as needed once trained)
model = create_cnn_architecture_model1(input_shape=INPUT_SHAPE)
model.load_weights('model_weights/cnn_mnist_digits_wt.h5')

# predict and evaluate on test dataset
x_test_scaled = x_test_gr / 255.
predictions = model.predict(x_test_scaled)
y_prediction = np.argmax(predictions, axis=1)
print(classification_report(y_test, y_prediction, target_names=class_names))
pd.DataFrame(confusion_matrix(y_test, y_prediction), index=class_names, columns=class_names)

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       980
           1       1.00      1.00      1.00      1135
           2       0.99      1.00      0.99      1032
           3       0.99      1.00      0.99      1010
           4       0.99      0.99      0.99       982
           5       0.98      0.99      0.99       892
           6       1.00      0.99      0.99       958
           7       0.99      0.99      0.99      1028
           8       0.99      0.99      0.99       974
           9       0.99      0.98      0.98      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000



,0,1,2,3,4,5,6,7,8,9
0,978,0,0,0,0,0,0,0,1,1
1,0,1130,2,2,0,0,0,1,0,0
2,0,0,1027,0,1,0,0,4,0,0
3,0,0,1,1006,0,2,0,0,1,0
4,0,0,0,0,970,0,0,1,1,10
5,1,0,0,8,0,882,1,0,0,0
6,3,1,2,0,3,2,944,0,3,0
7,0,1,4,0,0,0,0,1020,1,2
8,2,0,1,0,0,1,0,1,969,0
9,0,0,1,3,2,9,0,1,2,991
